<h3 style="text-align: center;"> Language Models Lab 3</h3>
<h5 style="text-align: center;"> Wenjie Hu 343312</h5>



Problem 1

In [1]:
! pip install torch lightning
! pip install sacremoses


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 10.0 MB/s eta 0:00:00


In [2]:
import os
import torch
import numpy as np
import pandas as pd
import random
from lightning.pytorch import seed_everything
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM
from sklearn.linear_model import LogisticRegression

In [3]:
def load_model(model_name="gpt2", device=None, causal=False):
    device = device or "cuda" if torch.cuda.is_available() else "cpu"
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    loader = AutoModelForCausalLM if causal else AutoModel
    model = loader.from_pretrained(model_name).to(device)
    return model, tokenizer, device

In [4]:
def ABX(wembedds: dict, clusters: dict, groups: list, words: list, N=500000):
    vectors = {}
    for word, embedding in wembedds.items():
        v = embedding.detach().cpu().numpy().squeeze()
        length = np.linalg.norm(v)
        vectors[word] = v / length

    def choose_another(pool, current):
        while True:
            choice = random.choice(pool)
            if choice != current:
                return choice

    missing_words = [word for word in words if word not in vectors]
    print("Missing words:", " ".join(missing_words))
    print("PROBLEMS:", len(missing_words) / len(words))

    print("Start evaluation")

    score = 0.0

    for _ in range(N):
        g1 = random.choice(groups)
        A = clusters[g1]
        B = clusters[choose_another(groups, g1)]

        a1 = random.choice(A)
        a2 = choose_another(A, a1)
        b = random.choice(B)

        if any(word not in vectors for word in (a1, a2, b)):
            score += 0.5
            continue

        va1, va2, vb = vectors[a1], vectors[a2], vectors[b]

        if np.dot(va1, va2) > np.dot(va1, vb):
            score += 1.0
        else:
          pass
    final_score = score / N
    print("TOTAL SCORE:", final_score)
    return final_score


In [ ]:
bert, bert_tokenizer, device = load_model(model_name="allegro/herbert-base-cased")
papuga, papuga_tokenizer, device = load_model(model_name="flax-community/papuGaPT2", causal=True)

In [6]:
def papuga_word_embedding(word):
  def papuga_tokenize(word):
    ids = papuga_tokenizer(word, return_tensors='pt')['input_ids'][0]
    return [papuga_tokenizer.decode(n) for n in ids]

  emb = papuga.transformer.wte.weight.detach().cpu().numpy()

  tokens = papuga_tokenize(word)
  return torch.tensor(emb[[papuga_tokenizer.encode(token) for token in tokens]], dtype=float).mean(dim=0)

In [7]:
def bert_word_embedding(word):
    input_ids = bert_tokenizer(word, return_tensors='pt')['input_ids'] #.to(device)
    output = bert(input_ids=input_ids)
    return output.last_hidden_state[0,0,:]

In [8]:
clusters_txt = '''
piśmiennicze: pisak flamaster ołówek długopis pióro
małe_ssaki: mysz szczur chomik łasica kuna bóbr
okręty: niszczyciel lotniskowiec trałowiec krążownik pancernik fregata korweta
lekarze: lekarz pediatra ginekolog kardiolog internista geriatra
zupy: rosół żurek barszcz
uczucia: miłość przyjaźń nienawiść gniew smutek radość strach
działy_matematyki: algebra analiza topologia logika geometria
budynki_sakralne: kościół bazylika kaplica katedra świątynia synagoga zbór
stopień_wojskowy: chorąży podporucznik porucznik kapitan major pułkownik generał podpułkownik
grzyby_jadalne: pieczarka borowik gąska kurka boczniak kania
prądy_filozoficzne: empiryzm stoicyzm racjonalizm egzystencjalizm marksizm romantyzm
religie: chrześcijaństwo buddyzm islam prawosławie protestantyzm kalwinizm luteranizm judaizm
dzieła_muzyczne: sonata synfonia koncert preludium fuga suita
cyfry: jedynka dwójka trójka czwórka piątka szóstka siódemka ósemka dziewiątka
owady: ważka biedronka żuk mrówka mucha osa pszczoła chrząszcz
broń_biała: miecz topór sztylet nóż siekiera
broń_palna: karabin pistolet rewolwer fuzja strzelba
komputery: komputer laptop kalkulator notebook
kolory: biel żółć czerwień błękit zieleń brąz czerń
duchowny: wikary biskup ksiądz proboszcz rabin pop arcybiskup kardynał pastor
ryby: karp śledź łosoś dorsz okoń sandacz szczupak płotka
napoje_mleczne: jogurt kefir maślanka
czynności_sportowe: bieganie skakanie pływanie maszerowanie marsz trucht
ubranie:  garnitur smoking frak żakiet marynarka koszula bluzka sweter sweterek sukienka kamizelka spódnica spodnie
mebel: krzesło fotel kanapa łóżko wersalka sofa stół stolik ława
przestępca: morderca zabójca gwałciciel złodziej bandyta kieszonkowiec łajdak łobuz
mięso_wędliny wieprzowina wołowina baranina cielęcina boczek baleron kiełbasa szynka schab karkówka dziczyzna
drzewo: dąb klon wiąz jesion świerk sosna modrzew platan buk cis jawor jarzębina akacja
źródło_światła: lampa latarka lampka żyrandol żarówka reflektor latarnia lampka
organ: wątroba płuco serce trzustka żołądek nerka macica jajowód nasieniowód prostata śledziona
oddziały: kompania pluton batalion brygada armia dywizja pułk
napój_alkoholowy: piwo wino wódka dżin nalewka bimber wiśniówka cydr koniak wiśniówka
kot_drapieżny: puma pantera lampart tygrys lew ryś żbik gepard jaguar
metal: żelazo złoto srebro miedź nikiel cyna cynk potas platyna chrom glin aluminium
samolot: samolot odrzutowiec awionetka bombowiec myśliwiec samolocik helikopter śmigłowiec
owoc: jabłko gruszka śliwka brzoskwinia cytryna pomarańcza grejpfrut porzeczka nektaryna
pościel: poduszka prześcieradło kołdra kołderka poduszeczka pierzyna koc kocyk pled
agd: lodówka kuchenka pralka zmywarka mikser sokowirówka piec piecyk piekarnik
'''
clusters = {}
words = set()
for x in clusters_txt.split("\n"):
    L = x.split()
    if len(L) < 2:
        continue
    clusters[L[0]] = L[1:]
    words.update(L[1:])
groups = list(clusters.keys())

In [9]:
papuga_emd = {word: papuga_word_embedding(word) for word in words}
ABX(papuga_emd, clusters, groups,words,N=50000)


Missing words: 
PROBLEMS: 0.0
Start evaluation
TOTAL SCORE: 0.58682


0.58682

In [10]:
bert_emb = {word: bert_word_embedding(word) for word in words}
ABX(bert_emb, clusters, groups,words,N=50000)


Missing words: 
PROBLEMS: 0.0
Start evaluation
TOTAL SCORE: 0.5911


0.5911

### neighbours of a key on keyboard
obtain the neighboring keys for each character, and then choice one key randomly from its neighbour and replaced by its neighbour *key*

In [11]:
import unicodedata

def remove_accents(input_str):
  nkfd_form = unicodedata.normalize('NFKD', input_str)
  no_accents = ''.join([c for c in nkfd_form if not unicodedata.combining(c)])
  return no_accents.replace('ł', 'l')

def get_keys_around(key):
    lines = 'qwertyuiop', 'asdfghjkl', 'zxcvbnm'
    line_index, index = [(i, l.find(key)) for i, l in enumerate(lines) if key in l][0]
    lines = lines[line_index-1: line_index+2] if line_index else lines[0: 2]
    return [
        line[index + i] for line in lines for i in [-1, 0, 1]
        if len(line) > index + i and line[index + i] != key and index + i >= 0]

def random_qwerty_swap(word, swap_count=1):
    # Convert the word to a list of characters for mutability
    word_chars = list(word)
    indices = list(range(len(word_chars)))
    random.shuffle(indices)

    for _ in range(min(swap_count, len(word))):
        idx = indices.pop()  # Take a random index
        char = remove_accents(word_chars[idx].lower()) # convert polish letter to english letter
        # print(char)
        if char in get_keys_around(char):
            replacement = random.choice(get_keys_around(char))
            word_chars[idx] = replacement

    return ''.join(word_chars)

In [12]:
qwerty_emb = {word: bert_word_embedding(random_qwerty_swap(word)) for word in words}
ABX(qwerty_emb, clusters, groups,words,N=50000)


Missing words: 
PROBLEMS: 0.0
Start evaluation
TOTAL SCORE: 0.59178


0.59178

### character swap
randomly choice two letter within a given word, and swap the two letters

In [13]:
import random

def character_swap(word):
  # swaps two randomly selected characters within a given word.
  if len(word) < 2:
    return word
  idx1, idx2 = random.sample(range(len(word)), 2)
  word_list = list(word)
  word_list[idx1], word_list[idx2] = word_list[idx2], word_list[idx1]
  return "".join(word_list)

In [14]:
qwerty_emb = {word: bert_word_embedding(character_swap(word)) for word in words}
ABX(qwerty_emb, clusters, groups,words,N=50000)


Missing words: 
PROBLEMS: 0.0
Start evaluation
TOTAL SCORE: 0.54184


0.54184

# Problem 2

In [15]:
!wget https://github.com/wenjie-hoo/Uni_stuff/raw/refs/heads/main/Language_Models/lab3/reviews.txt
! ls

--2024-12-28 08:16:58--  https://github.com/wenjie-hoo/Uni_stuff/raw/refs/heads/main/Language_Models/lab3/reviews.txt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wenjie-hoo/Uni_stuff/refs/heads/main/Language_Models/lab3/reviews.txt [following]
--2024-12-28 08:16:58--  https://raw.githubusercontent.com/wenjie-hoo/Uni_stuff/refs/heads/main/Language_Models/lab3/reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41457 (40K) [text/plain]
Saving to: ‘reviews.txt’

reviews.txt         100%[===================>]  40.49K  --.-KB/s    in 0.01s   

2024-12-28 08:16:58 (3.63 MB/s) - ‘reviews.tx

In [16]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [17]:
def load_review_data(path="reviews.txt"):
  data = []
  with open(path, "r", encoding="utf-8") as file:
    for line in file:
      sentiment, review_text = line.strip().split(" ", 1)
      data.append({"label": sentiment == "GOOD", "text": review_text})

  return pd.DataFrame(data)
reviews_df = load_review_data('reviews.txt')

In [18]:
reviews_df

,label,text
0,True,"Lokalizacja w centrum - przy głównym deptaku, ..."
1,True,"Hotel jest położony prawie nad samym jeziorem,..."
2,True,Warto przejechać ponad 600 km żeby przez tydzi...
3,True,"Wypożyczalnia samochodów w firmie hotelowej, r..."
4,True,Butelkę mamy 3 miesiące i nic się ze smoczkiem...
...,...,...
395,False,Mała po tym dostała jeszcze większych kolek.
396,False,Jedyny minus to bardzo wysokie ceny jak na Pol...
397,False,Oczywiście nie obwiniam Doktora Hodery o to co...
398,False,Mało przyjemne.


In [19]:

#from herbert.ipynb
def bert_representation(txt):

    input_ids = bert_tokenizer(txt, return_tensors='pt')['input_ids']
    output = bert(input_ids=input_ids)
    return output.last_hidden_state.detach().cpu().numpy()[0,0,:]

def log_probs_from_logits(logits, labels):
    logp = torch.nn.functional.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label

def papuga_sentence_probability(txt):
    suffix = " Pozytywne opinie na ten temat."
    input_ids = papuga_tokenizer(txt + suffix, return_tensors='pt')['input_ids']
    with torch.no_grad():
        output = papuga(input_ids=input_ids)
        log_probs = log_probs_from_logits(output.logits[:, :-1, :], input_ids[:, 1:])
        seq_log_probs = torch.sum(log_probs)
    return seq_log_probs.cpu().numpy().item()

In [20]:
def extract_features(df):
	df = df.copy().join(df.text.apply(bert_representation).apply(pd.Series).add_prefix('features.bert.'))
	df["features.papuga.probability"] = df.text.map(papuga_sentence_probability)
	df.columns = pd.MultiIndex.from_tuples([col.split('.') for col in df.columns])
	return df

In [21]:
train_df, test_df = train_test_split(reviews_df, test_size=0.2, random_state=42, shuffle=True)

train_features_df = extract_features(train_df)
test_features_df = extract_features(test_df)

In [22]:
clf = LogisticRegression(max_iter=1000).fit(train_features_df.features.values, train_features_df.label.squeeze())
print ('Train accuracy:', clf.score(train_features_df.features.values, train_features_df.label.squeeze()))
print ('Test accuracy:', clf.score(test_features_df.features.values, test_features_df.label.squeeze()))


Train accuracy: 1.0
Test accuracy: 0.775


# Problem 3

### a) Mechanical augmentatio

In [23]:
reviews_df.text.head()

,text
0,"Lokalizacja w centrum - przy głównym deptaku, ..."
1,"Hotel jest położony prawie nad samym jeziorem,..."
2,Warto przejechać ponad 600 km żeby przez tydzi...
3,"Wypożyczalnia samochodów w firmie hotelowej, r..."
4,Butelkę mamy 3 miesiące i nic się ze smoczkiem...


### randomly capical the first letter for a word

In [24]:
import random

def random_case_first_letter(text):
    words = text.split()  # Split the text into words
    modified_words = [
        (word[0].upper() if random.choice([True, False]) else word[0].lower()) + word[1:]
        if word else word
        for word in words
    ]
    return ' '.join(modified_words)

reviews_df['text_random_case'] = reviews_df['text'].apply(random_case_first_letter)

print(reviews_df[['text', 'text_random_case']].head())


                                                text  \
0  Lokalizacja w centrum - przy głównym deptaku, ...   
1  Hotel jest położony prawie nad samym jeziorem,...   
2  Warto przejechać ponad 600 km żeby przez tydzi...   
3  Wypożyczalnia samochodów w firmie hotelowej, r...   
4  Butelkę mamy 3 miesiące i nic się ze smoczkiem...   

                                    text_random_case  
0  Lokalizacja W Centrum - przy Głównym Deptaku, ...  
1  Hotel jest położony Prawie Nad samym Jeziorem,...  
2  warto przejechać ponad 600 km żeby przez tydzi...  
3  wypożyczalnia samochodów W Firmie hotelowej, r...  
4  butelkę Mamy 3 Miesiące I nic się Ze Smoczkiem...  


In [25]:
reviews_df['text'] = reviews_df['text_random_case'].apply(random_case_first_letter)
print(reviews_df)
reviews_df.drop(columns=['text_random_case'], inplace=True)


     label                                               text  \
0     True  Lokalizacja W centrum - Przy Głównym deptaku, ...   
1     True  hotel Jest położony Prawie nad samym jeziorem,...   
2     True  warto Przejechać ponad 600 Km żeby Przez Tydzi...   
3     True  wypożyczalnia Samochodów W firmie Hotelowej, r...   
4     True  butelkę mamy 3 Miesiące i nic się ze smoczkiem...   
..     ...                                                ...   
395  False       Mała Po tym dostała Jeszcze Większych Kolek.   
396  False  Jedyny minus to Bardzo Wysokie Ceny jak Na Pol...   
397  False  oczywiście nie obwiniam Doktora hodery o To Co...   
398  False                                    mało przyjemne.   
399  False           może Wystrój nie do Końca W moim guście.   

                                      text_random_case  
0    Lokalizacja W Centrum - przy Głównym Deptaku, ...  
1    Hotel jest położony Prawie Nad samym Jeziorem,...  
2    warto przejechać ponad 600 km żeby przez ty

In [26]:
train_df, test_df = train_test_split(reviews_df, test_size=0.2, random_state=42, shuffle=True)

train_features_df = extract_features(train_df)
test_features_df = extract_features(test_df)

In [27]:
clf = LogisticRegression(max_iter=1000).fit(train_features_df.features.values, train_features_df.label.squeeze())
print ('Train accuracy:', clf.score(train_features_df.features.values, train_features_df.label.squeeze()))
print ('Test accuracy:', clf.score(test_features_df.features.values, test_features_df.label.squeeze()))


Train accuracy: 1.0
Test accuracy: 0.7875


### b) Augmentation with a generative model

In [ ]:
import pandas as pd
from transformers import pipeline

generator = pipeline('text-generation', model='flax-community/papuGaPT2')


def augment_review(review, num_augments=1):
    augmented_reviews = []
    for _ in range(num_augments):
        # Generate text
        generated = generator(
            review,
            max_new_tokens=50,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.7
        )
        print(generated)
        augmented_reviews.append(generated[0]['generated_text'])
    return augmented_reviews

num_augments_per_review = 3
augmented_data = []

for index, row in reviews_df.iterrows():
    original_review = row['text']
    label = row['label']

    augmented_reviews = augment_review(original_review, num_augments=num_augments_per_review)

    for aug_review in augmented_reviews:
        augmented_data.append({'label': label, 'text': aug_review})

augmented_df = pd.DataFrame(augmented_data)
augmented_df.to_csv('augmented_reviews.csv', index=False)

print("Augmentation complete. Saved to 'augmented_reviews.csv'.")


In [33]:
augmented_df

,label,text
0,True,"Lokalizacja W centrum - Przy Głównym deptaku, ..."
1,True,"Lokalizacja W centrum - Przy Głównym deptaku, ..."
2,True,"Lokalizacja W centrum - Przy Głównym deptaku, ..."
3,True,"hotel Jest położony Prawie nad samym jeziorem,..."
4,True,"hotel Jest położony Prawie nad samym jeziorem,..."
...,...,...
1195,False,"mało przyjemne.\n– Nie mam pojęcia, ile to pot..."
1196,False,mało przyjemne.\nW tym momencie mam już prawie...
1197,False,może Wystrój nie do Końca W moim guście.\nBard...
1198,False,może Wystrój nie do Końca W moim guście. Nie w...


In [34]:
train_df, test_df = train_test_split(augmented_df, test_size=0.2, random_state=42, shuffle=True)

train_features_df = extract_features(train_df)
test_features_df = extract_features(test_df)

In [37]:
clf = LogisticRegression(max_iter=5000).fit(train_features_df.features.values, train_features_df.label.squeeze())
print ('Train accuracy:', clf.score(train_features_df.features.values, train_features_df.label.squeeze()))
print ('Test accuracy:', clf.score(test_features_df.features.values, test_features_df.label.squeeze()))


Train accuracy: 0.996875
Test accuracy: 0.925


### Augmentation Word2Vec

In [47]:
! wget https://git.wmi.amu.edu.pl/s464871/sport-text-classification-ball/raw/branch/master/word2vec_100_3_polish.bin
! wget https://git.wmi.amu.edu.pl/s464871/sport-text-classification-ball/raw/branch/master/word2vec_100_3_polish.bin.syn0.npy
! ls

--2024-12-28 10:09:40--  https://git.wmi.amu.edu.pl/s464871/sport-text-classification-ball/raw/branch/master/word2vec_100_3_polish.bin.syn0.npy
Resolving git.wmi.amu.edu.pl (git.wmi.amu.edu.pl)... 150.254.78.40
Connecting to git.wmi.amu.edu.pl (git.wmi.amu.edu.pl)|150.254.78.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 773612128 (738M) [application/octet-stream]
Saving to: ‘word2vec_100_3_polish.bin.syn0.npy’

word2vec_100_3_poli  21%[===>                ] 160.69M   146KB/s    in 5m 25s  

2024-12-28 10:15:06 (506 KB/s) - Connection closed at byte 168497826. Retrying.

--2024-12-28 10:15:07--  (try: 2)  https://git.wmi.amu.edu.pl/s464871/sport-text-classification-ball/raw/branch/master/word2vec_100_3_polish.bin.syn0.npy
Connecting to git.wmi.amu.edu.pl (git.wmi.amu.edu.pl)|150.254.78.40|:443... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 773612128 (738M), 605114302 (577M) remaining [application/octet-stream]
Saving to: 

In [48]:
import gensim.downloader as api
import random
import pandas as pd
from gensim.models import KeyedVectors


# word2vec_model =KeyedVectors.load_word2vec_format("word2vec_100_3_polish.bin", binary=True)# Use pre-trained Google News embeddings
word2vec_model =KeyedVectors.load("word2vec_100_3_polish.bin")

def augment_text_with_word2vec(review, replacement_prob=0.3):
    words = review.split()
    augmented_words = []

    for word in words:
        # decide whether to replace this word
        if random.random() < replacement_prob and word.lower() in word2vec_model:
            # find most similar word
            try:
                similar_words = word2vec_model.most_similar(word.lower(), topn=5)
                replacement_word = random.choice(similar_words)[0]
                augmented_words.append(replacement_word)
            except KeyError:
                # if the word is not in vocabulary, keep it unchanged
                augmented_words.append(word)
        else:
            augmented_words.append(word)

    return " ".join(augmented_words)

def augment_reviews(dataframe, column_name, num_augments=3):
    augmented_data = []

    for index, row in dataframe.iterrows():
        original_review = row[column_name]
        label = row['label']
        for _ in range(num_augments):
            augmented_review = augment_text_with_word2vec(original_review)
            augmented_data.append({'label': label, 'text': augmented_review})

    return pd.DataFrame(augmented_data)

augmented_w2v_df = augment_reviews(reviews_df, column_name='text', num_augments=3)
augmented_w2v_df.to_csv('augmented_reviews_word2vec.csv', index=False)
print("Augmentation complete. Saved to 'augmented_reviews_word2vec.csv'.")


/usr/local/lib/python3.10/dist-packages/gensim/models/keyedvectors.py:849: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


Augmentation complete. Saved to 'augmented_reviews_word2vec.csv'.


In [49]:
augmented_w2v_df

,label,text
0,True,"Lokalizacja W centrum - nad Głównym deptaku, P..."
1,True,"Lokalizacja W centrum , Przy Głównym deptaku, ..."
2,True,"Lokalizacja W centrum - Przy Głównym deptaku, ..."
3,True,hotel wliczać położony Prawie nad samym jezior...
4,True,hotel znaczyć położony Prawie nad samym jezior...
...,...,...
1195,False,mało przyjemne.
1196,False,mało przyjemne.
1197,False,jeżeli Wystrój nie doktórego Końca W moim guście.
1198,False,może Wystrój nie doniej Końca ; moim guście.


In [51]:
train_df, test_df = train_test_split(augmented_w2v_df, test_size=0.2, random_state=42, shuffle=True)

train_features_df = extract_features(train_df)
test_features_df = extract_features(test_df)

In [52]:
clf = LogisticRegression(max_iter=5000).fit(train_features_df.features.values, train_features_df.label.squeeze())
print ('Train accuracy:', clf.score(train_features_df.features.values, train_features_df.label.squeeze()))
print ('Test accuracy:', clf.score(test_features_df.features.values, test_features_df.label.squeeze()))


Train accuracy: 0.99375
Test accuracy: 0.8791666666666667


### num_aug = 1

In [53]:
augmented_w2v_df_ = augment_reviews(reviews_df, column_name='text', num_augments=1)
augmented_w2v_df_.to_csv('augmented_reviews_word2vec.csv', index=False)


/usr/local/lib/python3.10/dist-packages/gensim/models/keyedvectors.py:849: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


In [54]:
augmented_w2v_df_

,label,text
0,True,"usytuowanie i trum - przed Głównym deptaku, Po..."
1,True,hotel Jest położony niemalże nad samym jeziore...
2,True,warto przejeżdżać ponad 500 kilometr żebym Prz...
3,True,"przechowalnia Samochodów . firmie Hotelowej, g..."
4,True,butelkę mamy 4 Miesiące i nic się ze smoczkiem...
...,...,...
395,False,mały Po taki dostała Jeszcze Większych Kolek.
396,False,niezastąpiony celsjusza to Bardzo Wysokie Ceny...
397,False,oczywiście nie obwiniam Doktora hodery o - cóż...
398,False,mało przyjemne.


In [55]:
train_df, test_df = train_test_split(augmented_w2v_df_, test_size=0.2, random_state=42, shuffle=True)

train_features_df = extract_features(train_df)
test_features_df = extract_features(test_df)

In [56]:
clf = LogisticRegression(max_iter=1000).fit(train_features_df.features.values, train_features_df.label.squeeze())
print ('Train accuracy:', clf.score(train_features_df.features.values, train_features_df.label.squeeze()))
print ('Test accuracy:', clf.score(test_features_df.features.values, test_features_df.label.squeeze()))


Train accuracy: 0.996875
Test accuracy: 0.7375
